In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

Napravljen je pandas DataFrame gde su atributi zasebno pokrenute putanje, dok ce ciljna promenljiva da bude srednja vrednost svih putanja

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [14]:
percent_val = 0.2

In [15]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

In [16]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))

In [17]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in test_file_names], axis=0)

In [18]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [19]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

In [20]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*61000+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [21]:
for j in range(len(test_file_names)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(x_command[i] - test_[j*61000+i])")
    exec(f'razlika_test_{j} = np.asarray(razlika_test_{j})')    

In [24]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [25]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for i in range(61000):    
    for j in range(len(train_files_names)):
        exec(f'current_file = razlika_trening_{j}')
        if current_file[i][0] < min_kord[i]:
            min_kord[i] = current_file[i][0]
        if current_file[i][0] > max_kord[i]:
            max_kord[i] = current_file[i][0]

In [26]:
time_steps = 8
input_shape = (time_steps,1)

In [19]:
time_steps_list = [8,10]

In [21]:
best_step = None
best_skor = float('-inf')

for step in time_steps_list:
    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')
    for i in range(len(test_file_names)):
        exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, step)')
        
    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')
        
    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 
    
    test_X = np.asarray(test_X_0)
    for i in range(1,len(test_file_names)):
        exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')
    
    test_y = np.asarray(test_y_0)
    for i in range(1,len(test_file_names)):
        exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

    input_shape = (step,1)
    
    keras.utils.set_random_seed(7)
    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
    modelMSE = Sequential()
    modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
    #modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
    modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)
    
    historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 64, verbose = 2, shuffle = False)
    #train_scores = modelMSE.evaluate(test_X, test_y)
    test_predict_MSE = modelMSE.predict(test_X)
    
    uspeli_minmax = 0
    for j in range(len(test_file_names)):
        for i in range(int(len(test_predict_MSE)/len(test_file_names))):
            if min_kord[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= max_kord[i+time_steps]:
                uspeli_minmax += 1
    
    if uspeli_minmax > best_skor:
        best_skor = uspeli_minmax
        best_step = step
        
        print('best_skor', best_skor, 'best_step',best_step)

print('best_skor', best_skor)
print('best_step', best_step)


Epoch 1/10

22872/22872 - 68s - loss: 4.9113 - 68s/epoch - 3ms/step
Epoch 2/10
22872/22872 - 65s - loss: 2.2804 - 65s/epoch - 3ms/step
Epoch 3/10
22872/22872 - 55s - loss: 2.1692 - 55s/epoch - 2ms/step
Epoch 4/10
22872/22872 - 55s - loss: 2.1119 - 55s/epoch - 2ms/step
Epoch 5/10
22872/22872 - 54s - loss: 2.0642 - 54s/epoch - 2ms/step
Epoch 6/10
22872/22872 - 51s - loss: 2.0256 - 51s/epoch - 2ms/step
Epoch 7/10
22872/22872 - 52s - loss: 1.9920 - 52s/epoch - 2ms/step
Epoch 8/10
22872/22872 - 55s - loss: 1.9628 - 55s/epoch - 2ms/step
Epoch 9/10
22872/22872 - 54s - loss: 1.9397 - 54s/epoch - 2ms/step
Epoch 10/10
22872/22872 - 54s - loss: 1.9181 - 54s/epoch - 2ms/step
11436/11436 [==============================] - 17s 1ms/step
best_skor 312152 best_step 8
Epoch 1/10
22872/22872 - 83s - loss: 5.2565 - 83s/epoch - 4ms/step
Epoch 2/10
22872/22872 - 72s - loss: 2.3516 - 72s/epoch - 3ms/step
Epoch 3/10
22872/22872 - 86s - loss: 2.2094 - 86s/epoch - 4ms/step
Epoch 4/10
22872/22872 - 90s - loss: 2

In [27]:
time_steps = 8
input_shape = (time_steps,1)

In [28]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')
for i in range(len(test_file_names)):
    exec(f'test_X_{i}, test_y_{i} = create_timeseries_split(razlika_test_{i}, time_steps)')

train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 

test_X = np.asarray(test_X_0)
for i in range(1,len(test_file_names)):
    exec(f'test_X = np.append(test_X, test_X_{i}, axis=0)')

test_y = np.asarray(test_y_0)
for i in range(1,len(test_file_names)):
    exec(f'test_y = np.append(test_y, test_y_{i}, axis=0)')

In [49]:
val_x = train_X[-int(len(train_X)*0.2)-1:-1]
val_y = train_y[-int(len(train_X)*0.2)-1:-1]

In [72]:
val_x.shape[0]

292761

In [66]:
train_X = train_X[:val_x.shape[0]]
train_y = train_y[:val_y.shape[0]]

# TESTIRANJE ARHITEKTURE i LR/BATCH

In [101]:
gru_unit = [8,16]
unit1 = [8,16]
unit2 = [8,16]
LR = [0.01,0.001]
batch_size = [32,64]

In [115]:
best_skor = float('-inf')
best_gru = None
best_unit1 = None
best_unit2 = None
#best_batch = None
#best_LR = None

for gru in gru_unit:
    for uni1 in unit1:
        for uni2 in unit2:
            #for rate in LR:
            #for batch in batch_size:

            keras.utils.set_random_seed(7)
            optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

            model = Sequential()
            model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
            model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
            model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
            model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
            model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

            model.fit(train_X, train_y, epochs = 10, batch_size = 64, verbose = 2, shuffle = False)
            test_predict_MSE = model.predict(val_x)
            
#             uspeli_minmax = 0
#             for j in range(len(test_file_names)):
#                 for i in range(int(len(test_predict_MSE)/len(test_file_names))):
#                     if min_kord[i+time_steps] <= test_predict_MSE[int(len(test_predict_MSE)/len(test_file_names))*j + i] <= max_kord[i+time_steps]:
#                         uspeli_minmax += 1
            
            i = 8
            uspeli_minmax = 0
            for j in range(len(test_predict_MSE)):
                if i%61000 == 0:
                    i = 8
                if min_kord[i] <= test_predict_MSE[j] <= max_kord[i]:
                    uspeli_minmax += 1
                i += 1

            
            
            if uspeli_minmax > best_skor:
                best_skor = uspeli_minmax
                best_gru = gru
                best_unit1 = uni1
                best_unit2 = uni2
                #best_LR = rate
                #best_batch = batch

                print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
#print('best learning rate: ', best_LR)    
#print('best batch size: ', best_batch)    


Epoch 1/10
18298/18298 - 60s - loss: 9.9657 - 60s/epoch - 3ms/step
Epoch 2/10
18298/18298 - 54s - loss: 2.7851 - 54s/epoch - 3ms/step
Epoch 3/10
18298/18298 - 52s - loss: 2.5467 - 52s/epoch - 3ms/step
Epoch 4/10
18298/18298 - 51s - loss: 2.4423 - 51s/epoch - 3ms/step
Epoch 5/10
18298/18298 - 50s - loss: 2.3723 - 50s/epoch - 3ms/step
Epoch 6/10
18298/18298 - 51s - loss: 2.3055 - 51s/epoch - 3ms/step
Epoch 7/10
18298/18298 - 53s - loss: 2.2565 - 53s/epoch - 3ms/step
Epoch 8/10
18298/18298 - 53s - loss: 2.2270 - 53s/epoch - 3ms/step
Epoch 9/10
18298/18298 - 53s - loss: 2.2036 - 53s/epoch - 3ms/step
Epoch 10/10
18298/18298 - 53s - loss: 2.1839 - 53s/epoch - 3ms/step
9149/9149 [==============================] - 14s 1ms/step
best_skor:  169032 best_gru:  8 best_unit1:  8 best_unit2:  8
Epoch 1/10
18298/18298 - 58s - loss: 10.4502 - 58s/epoch - 3ms/step
Epoch 2/10
18298/18298 - 55s - loss: 2.9156 - 55s/epoch - 3ms/step
Epoch 3/10
18298/18298 - 54s - loss: 2.4645 - 54s/epoch - 3ms/step
Epoch 4

In [116]:
len(test_predict_MSE)

292761

In [117]:
batch_size = [64,128,256,512]

In [119]:
best_skor_batch = float('-inf')
best_batch = None

for batch in batch_size:
    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
    keras.utils.set_random_seed(7)
    
    model = Sequential()
    model.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
    model.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    model.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer = optimizerMSE)
    
    model.fit(train_X, train_y, epochs=10, batch_size = batch, verbose = 2)
    #train_scores = model.evaluate(test_X, test_y)
    test_predict_MSE = model.predict(val_x)
    i = 8
    uspeli_minmax = 0
    for j in range(len(test_predict_MSE)):
        if i%61000 == 0:
            i = 8
        if min_kord[i] <= test_predict_MSE[j] <= max_kord[i]:
            uspeli_minmax += 1
        i += 1
    
    if uspeli_minmax > best_skor_batch:
        best_skor_batch = uspeli_minmax
        best_batch = batch
print('best_skor: ', best_skor_batch)
print('best batch size: ', best_batch)    

Epoch 1/10
18298/18298 - 65s - loss: 5.8070 - 65s/epoch - 4ms/step
Epoch 2/10
18298/18298 - 55s - loss: 2.5542 - 55s/epoch - 3ms/step
Epoch 3/10
18298/18298 - 55s - loss: 2.3393 - 55s/epoch - 3ms/step
Epoch 4/10
18298/18298 - 55s - loss: 2.2545 - 55s/epoch - 3ms/step
Epoch 5/10
18298/18298 - 55s - loss: 2.1963 - 55s/epoch - 3ms/step
Epoch 6/10
18298/18298 - 54s - loss: 2.1544 - 54s/epoch - 3ms/step
Epoch 7/10
18298/18298 - 54s - loss: 2.1215 - 54s/epoch - 3ms/step
Epoch 8/10
18298/18298 - 54s - loss: 2.0970 - 54s/epoch - 3ms/step
Epoch 9/10
18298/18298 - 54s - loss: 2.0770 - 54s/epoch - 3ms/step
Epoch 10/10
18298/18298 - 54s - loss: 2.0596 - 54s/epoch - 3ms/step
9149/9149 [==============================] - 14s 2ms/step
Epoch 1/10
9149/9149 - 33s - loss: 7.8209 - 33s/epoch - 4ms/step
Epoch 2/10
9149/9149 - 30s - loss: 2.8426 - 30s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 30s - loss: 2.4959 - 30s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 30s - loss: 2.3822 - 30s/epoch - 3ms/step
Epoch 5/10


In [67]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [40]:
historyMAE = modelMAE.fit(train_X, train_y, epochs=10, batch_size = 16, verbose = 2)

NameError: name 'modelMAE' is not defined

In [68]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 1024, verbose = 2)

Epoch 1/10
894/894 - 9s - loss: 7173579776.0000 - 9s/epoch - 10ms/step
Epoch 2/10
894/894 - 2s - loss: 48031884.0000 - 2s/epoch - 3ms/step
Epoch 3/10
894/894 - 2s - loss: 1464.7377 - 2s/epoch - 2ms/step
Epoch 4/10
894/894 - 2s - loss: 1464.3621 - 2s/epoch - 2ms/step
Epoch 5/10
894/894 - 2s - loss: 1464.1544 - 2s/epoch - 2ms/step
Epoch 6/10
894/894 - 2s - loss: 1464.0243 - 2s/epoch - 2ms/step
Epoch 7/10
894/894 - 2s - loss: 1463.9316 - 2s/epoch - 3ms/step
Epoch 8/10
894/894 - 2s - loss: 1463.8832 - 2s/epoch - 3ms/step
Epoch 9/10
894/894 - 2s - loss: 1463.6785 - 2s/epoch - 2ms/step
Epoch 10/10
894/894 - 2s - loss: 1463.3251 - 2s/epoch - 2ms/step


In [69]:
#train_scores_MAE = modelMAE.evaluate(test_X, test_y)
train_scores_MSE = modelMSE.evaluate(test_X,test_y)

28594/28594 [==============================] - 37s 1ms/step - loss: 1463.4017


In [70]:
test_predict_MSE = modelMSE.predict(test_X)
#test_predict_MAE = modelMAE.predict(test_X)

28594/28594 [==============================] - 36s 1ms/step


# TESTIRANJE SA VISE INSTANCI